# Reoccurance Notebook 
* How often to we return to the same census tract for Sanitation Requetss 

In [1]:
import intake 
import pandas as pd 
import geopandas as gpd 
import altair as alt

In [2]:
catalog = intake.open_catalog("../catalogs/*.yml")

In [3]:
df = catalog.care311.read()

In [4]:
print(f"there have been {len(df)} requests since {df.createddate.min()}")

there have been 104809 requests since 2015-08-05T10:37:55.000


In [5]:
print(f"of those, {len(df[~df['closeddate'].isnull()])} have been closed")

of those, 94289 have been closed


In [6]:
df['closeddate'] = pd.to_datetime(df.closeddate)
df['createddate'] = pd.to_datetime(df.createddate)

## Create a solve time

In [7]:
## time to solve
df.solve_time = df[['createddate', 'closeddate']].diff(axis=1)['closeddate']

/opt/conda/lib/python3.7/site-packages/geopandas/geodataframe.py:97: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  super(GeoDataFrame, self).__setattr__(attr, val)


## Make a chart

In [8]:
df = df.set_index('closeddate')
source = pd.DataFrame(df.groupby([pd.Grouper(freq='M'), 'createdbyuserorganization']).count()['srnumber'])

In [9]:
chart = alt.Chart(source.reset_index()).mark_bar().encode(
    x = alt.X('closeddate:O'),
    y= alt.Y('sum(srnumber)'),
    color='createdbyuserorganization'
).properties(
    title='Number of Homelessness Encampment SRs closed by Month'
)

chart.encoding.y.title = 'Number of SR Encampments'
chart.encoding.x.title = 'Month'
chart

alt.Chart(...)

In [10]:
tracts = catalog.census_tracts_with_pop.read()

In [11]:
df = df[~df.geometry.isnull()]
print(f"length of DF with no nulls is {len(df)}")
df = df.to_crs(epsg=2229)

length of DF with no nulls is 104738


In [12]:
merged = gpd.sjoin(df, tracts, how="inner", op='intersects')
print(f'length of merged df is {len(merged)}')

/opt/conda/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!({'init': 'epsg:2229', 'no_defs': True} != {'init': 'epsg:2229'})
  "(%s != %s)" % (left_df.crs, right_df.crs)


length of merged df is 104505


# Analysis 
* now that the dataset is merged, we want to calculate how long it takes for a ticket, on average, to get opened up again in the same tract
* There are two metrics we want to track here 1) `closed->next closure` and then `closed->next opened` 

In [13]:
## closed to next closure
# drop computed column if exists 
if 'closed2closed' in merged:
    merged = merged.drop('closed2closed', axis =1)

def compute_stat(group):
    """
    return the series and use in the apply 
    """
    return group.index.to_series().diff(periods=-1)
    
grouped = merged.groupby('GEOID10', as_index="False")

s = pd.Series(grouped.apply(compute_stat))

In [14]:
closed2closed = s.reset_index().rename({'level_1': 'closeddate', 0: 'closed2closed'}, axis=1)

In [15]:
#merged = pd.merge(merged.reset_index().rename({'level_0': 'closeddate'}, axis=1), closed2closed)

In [16]:
print(f'the mean between is {merged.closed2closed.mean()}')
print(f'the median between is {merged.closed2closed.median()}')

AttributeError: 'GeoDataFrame' object has no attribute 'closed2closed'

In [ ]:
merged = merged.reset_index().rename({'level_0': 'closeddate'}, axis=1)

In [ ]:
# compute closed to next open 

raw_data = df.reset_index()
def compute_next_open(row):
    """
    For each row, find the next opened request in the df. 
    """
    date = row.loc['closeddate']
    geoid = row.loc['GEOID10']
    x = merged[(merged.createddate > date) & (merged.GEOID10 == geoid)]
    next_date = x.createddate.min()
    if next_date: 
        return next_date - date
    else: 
        return None


In [ ]:
s = merged.dropna(subset=['closeddate']).apply(compute_next_open, axis=1)
print(s)

In [ ]:
print(f'the average closed2open time is {s.mean()}')
print(f'the median closed2open time is {s.median()}')

In [ ]:
merged['closed2open'] = s

## Rolling Average Chart

In [ ]:
source = pd.DataFrame(merged.groupby([pd.Grouper(freq='W'), 'createdbyuserorganization']).mean())

In [ ]:
merged['closed2open'].rolling(window=2).mean()

In [ ]:
merged.GEO10.value_counts()